<h1> <center> Predicting the NHL Playoffs Using a Bagged Elastic Net Regression </center></h1> <br>

This is straight up a bootstrap aggregated elastic net GLM that has objective function:

$$log(L(p)) - \lambda\left[\frac{{1-\alpha}}{2} \sum_{j = 1}^p B_{j}^2 + \alpha \sum_{j = 1}^p \lvert B_{j}\rvert \right]$$ <br>

where $log(L(p))$ is the loglikelihood function of the binomial distribution (in this case of a binary classification) and  $B_{j}$  are the fitted coefficients for each feature. We tune over values of $\lambda$ and $\alpha$ to get the best out of sample AUROC. Note that $log[L(p)] = \sum_{i=1}^n [y_{i} * log(p_{i}) + (1-y_{i}) * log(1-p_{i})]$ but $p_{i} = \frac{exp(-X_{i}B)}{1 + exp(-X_{i}B)}$ which is clearly not trivial to solve.

I chose this model mostly because I wanted a framework that had embedded feature selection while still representing predictions as a linear combination of features. The sample size is low, so a "simple" linear function is probably the best we can get right now. From the loss function, the model explicitly controls for both collinearity in predictors (large amounts in this problem) through the ridge $\sum_{j = 1}^p B_{j}^2$ and noisy predictors through the LASSO  $\sum_{j = 1}^p \lvert B_{j}\rvert$ . $\alpha$ controls the mixing amount of the two penalties and $\lambda$ controls the entire contribution of penalty from both the ridge and LASSO. It should be clear that the model penalizes large coefficients, a symptom of bad collinearity or perfect separation in any linear regression while also allowing for the coefficient of a predictor to be exactly zero if the predictor is not useful in maximizing out of sample performance measures. A model that maximizes the log likelihood is not necessarily the best model if it is overfit; in some ways this is similar to the overall idea of Akaike's Information Criterion for finding a parsimonious model but the penalty term is very different. One easy way to interpret this model is as follows: if the loglikelihood function does not increase enough (or at all) to warrent a relatively "large"  $B_{j}$, then the influence the variable has on the predictions from the model is lowered by shrinking the variable's corresponding coefficient (in magnitude). In some cases, if the variable is completely non-predictive than the coefficient for that variable is shrunk to zero, meaning that the variable is straight up not included in the model anymore. Furthermore, coefficients that are arbitrarily (and not meaningfully) large due to collinearity/perfect separation are shrunk to smaller values to decrease their arbitrary large influence on predictions, reducing the variance of out of sample predictions (and therefore improving out of sample predictive performance).

The model is the best out of all models tried giving an AUROC of about 0.675 using about 140 features as input (though many aren't actually used in final predictions). It predicts slightly better than the bagged gradient boosted GLM that I tried. This means that in general, if we were to randomly pick a true winner and a true loser from the dataset 1000 times, the bagged elastic net model would be expected to rank the true winner as having a higher probability of being a winner over the true loser about 675 out of 1000 times. Not bad for how hard NHL games are to predict; the best game to game NHL models I've seen evaluate around 0.6709 log loss; that is, on average a probability of about 0.511 is being predicted for a true winner/true loser to be an actual winner/loser which is clearly not a very confident prediction. This is better than simply randomly guessing (AUROC = 0.5, or Log Loss of 0.6931 = ln2) or defaulting to selecting the higher seed in the playoffs (should give an AUROC of around 0.57 ish in that case).

The script takes kind of a while to run since we are doing 150 repeats. Be warned that this script is multithreaded; so your CPU will be pinned if you run it.

<h3><center>Reported Validation Scores (150 Repeats of Nested Cross Validation): </center></h3>
Final AUROC: 0.67512012012012 <br>
A 95% CI for the AUROC is: [0.672597256743428, 0.677642983496813] <br>


In [1]:
#Set the directory for parallel computation status checks. Change this to any folder on your computer so that we can monitor 
#the status of the repeated cross validation.
setwd("C:/Users/Brayden/Documents/NHLModel/Status")

In [2]:
#Dependencies

require(glmnet)
require(caret)
require(pROC)
require(tidyverse)
require(recipes)
require(moments)
require(doParallel)
require(foreach)
require(fastknn)

Loading required package: glmnet
Loading required package: Matrix
Warning message:
"package 'Matrix' was built under R version 3.5.2"Loading required package: foreach
Loaded glmnet 2.0-16

Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.5.2"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.5.2"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.5.1"Loading required package: pROC
Warning message:
"package 'pROC' was built under R version 3.5.2"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following object is masked from 'package:glmnet':

    auc

The following objects are masked from 'package:stats':

    cov, smooth, var

Loading required package: tidyverse
Warning message:
"package 'tidyverse' was built under R version 3.5.2"-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v tibble  1.4.2 

This is the main modelling function. We bag 15 elastic net models.

In [3]:
#..................................Bagging Function...................................#
baggedModel = function(train, test, label_train, alpha.a, s_lambda.a){
  
  set.seed(40689)
  samples = caret::createResample(y = label_train, times = 15)
  pred = vector("list", length(samples))
  varImp = vector("list", length(samples))
  
  for (g in 1:length(samples)){
    train_temp = train[samples[[g]], ]
    a = label_train[samples[[g]]]
    modelX = glmnet(x = data.matrix(train_temp[, !names(train_temp) %in% c("ResultProper")]), y = a, family = "binomial", alpha = alpha.a, nlambda = 120, standardize = FALSE)
    pred[[g]] = predict(modelX, newx = data.matrix(test[, !names(test) %in% c("ResultProper")]), type = "response")[, s_lambda.a]
    varImp[[g]] = varImp(modelX, lambda = modelX$lambda[s_lambda.a])
    colnames(varImp[[g]])[1] = paste("Overall:", g, sep = "")
    remove(modelX, train_temp, a)
  }
  
  pred = pred %>% Reduce(function(x,y) cbind(x,y),.) %>% as_tibble() %>%
    mutate(Predicted = rowMeans(.))
  
  varImp = varImp %>% Reduce(function(x,y) cbind(x,y),.) %>% as_tibble() %>%
    mutate(VariableImportance = rowMeans(.))
  
                             
  varImp = tibble::rownames_to_column(cbind.data.frame(meanImportance = varImp$VariableImportance), var = "Variable")
  
  list(Predictions = pred$Predicted, VariableImportance = varImp)
}

The log loss function which is not used currently. This can be easily added in if needed.

In [4]:
#..................................Log Loss Function....................................#

logLoss = function(scores, label){
  
  if (is.factor(label)){
    u = ifelse(label ==  "W", 1,0)
  } else{
    u = label
  }
  
  tmp = data.frame(scores = scores, target = u)
  tmp = tmp %>% mutate(scores = ifelse(scores == 1, 0.9999999999999999, ifelse(scores == 0 , 0.0000000000000001, scores))) %>%
    mutate(logLoss = -(target * log(scores) + (1-target) * log(1-scores)))
  
  mean(tmp$logLoss)
}


The PCA function used to create PCA variables. I did this as a function because I wanted to keep the original variables fed into the transformation, as well as the principle components.

In [5]:
#..................................PCA Function....................................#

addPCA_variables = function(traindata, testdata){
    
    traindata_tmp = traindata[, !names(traindata) %in% c("ResultProper")] %>% select_if(., is.numeric)
    testdata_tmp = testdata[, !names(testdata) %in% c("ResultProper")] %>% select_if(., is.numeric)
    
    pca_parameters = prcomp(traindata_tmp, center = FALSE, scale. = FALSE)
    pca_newdata = predict(pca_parameters, newdata = testdata_tmp)[,1:5]
    pca_traindata = predict(pca_parameters, newdata = traindata_tmp)[,1:5]
    list(train = cbind(traindata, pca_traindata), test = cbind(testdata, pca_newdata))
}

The kNN function is not used because it did not improve results on an older dataset. Perhaps it will now?

In [6]:
#..................................kNN Function....................................#

addKNN_variables = function(traindata, testdata, include_PCA = FALSE){
    
    y = traindata$ResultProper
    traindata = traindata[, !names(traindata) %in% c("ResultProper")]
    testdata = testdata[, !names(testdata) %in% c("ResultProper")]

    if(include_PCA == TRUE){
        
    traindata_tmp = traindata %>% select_if(., is.numeric)
    testdata_tmp = testdata %>% select_if(., is.numeric)
        
        }else{
  
    traindata_tmp = traindata %>% select_if(., is.numeric) %>% as_tibble(.) %>% select(-starts_with("PC"))
    testdata_tmp = testdata %>% select_if(., is.numeric) %>% as_tibble(.) %>% select(-starts_with("PC"))
    
    }
    
    newframeswithKNN = fastknn::knnExtract(xtr = data.matrix(traindata_tmp), ytr = y, xte = data.matrix(testdata_tmp), k = 1)
    KNN_train = newframeswithKNN$new.tr %>% as_tibble(.) %>% transmute_all(., .funs = function(x) (x - mean(x, na.rm=TRUE)) / sd(x, na.rm=TRUE))
    KNN_test = newframeswithKNN$new.te %>% as_tibble(.) %>% transmute_all(., .funs = function(x) (x - mean(x, na.rm=TRUE)) / sd(x, na.rm=TRUE)) 
    list(train = cbind(traindata, KNN_train), test = cbind(testdata, KNN_test))
}

The data represents differences in a stat from the first seeds perspective. This way we get a single prediction per a series. There could be better ways to do this, perhaps a ratio, product, sum, etc. I thought a difference would make the most sense.

Make sure to change the directories to pull in data from wherever the "Required Data Sets" folder is located. The "Required Data Sets" folder is in the GitHub repository.

In [7]:
#..................................Read data in....................................#
#Change directories to pull in data from the "Required Data Sets" folder located in the repository.

cat("Reading in Data..... \n")
allData = read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/HockeyReference2.csv") %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/HockeyReference1.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/CorsicaAllTeamStats.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/CorsicaGameScoreStats.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/ELORatings_January25_2019.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/ESPNStats.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/FenwickScores.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/NHLOfficialStatsJanuary25th.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/SCFScores_Feb3rd_2019.csv")) %>%
              bind_cols(read_csv("C:/Users/Brayden/Documents/GitHub/NHLPlayoffs/Required Data Sets/VegasOddsOpening.csv")) %>%
              mutate(ResultProper = as.factor(ResultProper))

Reading in Data..... 


Parsed with column specification:
cols(
  ResultProper = col_character(),
  PlayerPoints = col_double(),
  WeightedGoalieSavePercntage = col_double(),
  WeightedGPS = col_double(),
  H2H = col_double(),
  Q1Record = col_double(),
  Q2Record = col_double(),
  Q3Record = col_double(),
  Q4Record = col_double(),
  SDRecord = col_double()
)
Parsed with column specification:
cols(
  AverageAge = col_double(),
  GoalDiff = col_double(),
  GoalsAgainst = col_double(),
  GoalsFor = col_double(),
  OTL = col_double(),
  PenaltyKillPercentage = col_double(),
  PenaltyMinsPG = col_double(),
  Points = col_double(),
  PointsPercentageofMax = col_double(),
  PowerPlayGoals = col_double(),
  PowerPlayOppurtunities = col_double(),
  PowerPlayPercentage = col_double(),
  RegularSeasonWinPercentage = col_double(),
  ShotPercentage = col_double(),
  SOG = col_double(),
  SOS = col_double(),
  SRS = col_double()
)
Parsed with column specification:
cols(
  CF._Team = col_double(),
  CF_Per60Team = col_dou

In [8]:
#...................................Engineering of some features..................#

allData = allData %>% mutate(Round = as.factor(rep(c(1,1,1,1,1,1,1,1,2,2,2,2,3,3,4),13))) %>%
            mutate(Ratio_of_GoalstoGoalsAgainst = GoalsFor/GoalsAgainst) %>%
            mutate(Ratio_of_HitstoBlocks = HitsatES/BlocksatES) %>%
            mutate(logofPoints = sign(Points) * log(abs(Points) + 1)) %>%
            mutate(sqrtofPoints = abs(Points)^0.5) %>%
            mutate(PenaltyMinstoPowerPlay = PenaltyMinsPG*60*82 /PowerPlayPercentage) %>%
            mutate(Ratio_of_SRStoPoints = SRS/Points) %>%
            mutate(AverageGoalDiff_PerGame = GoalsFor/82) %>%
            mutate(AveragePenaltyDiff_PerGame = PenaltyMinsPG/82) %>%
            mutate(logofSOG = sign(SOG) * log(abs(SOG) + 1)) %>%
            mutate(sqrtofRPI = abs(RPI)^0.5) %>%
            mutate(PowerPlaytoPenaltyKill = PowerPlayPercentage/PenaltyKillPercentage) %>%
            mutate(PowerPlaytoPenaltyKill = sign(PowerPlaytoPenaltyKill) * log(abs(PowerPlaytoPenaltyKill) + 1)) %>%
            mutate(SCFtoGoalsAgainst = SCF/GoalsAgainst) %>%
            mutate(PointsPercentage = Points/164) %>%
            mutate(GS_max_log = sign(GS_mean) * log(abs(GS_mean) + 1)) %>%
            mutate(CA_Per60Team_log = sign(CA_Per60Team) * log(abs(CA_Per60Team) + 1)) %>%
            mutate(Ratio_of_GoalstoGoalsAgainstlog = sign(Ratio_of_GoalstoGoalsAgainst) * log(abs(Ratio_of_GoalstoGoalsAgainst) +1)) %>%
            mutate(Ratio_of_HitstoBlockslog = sign(Ratio_of_HitstoBlocks) * log(abs(Ratio_of_HitstoBlocks) + 1)) %>%
            mutate(SCFtoGoalsAgainstlog = sign(SCFtoGoalsAgainst) * log(abs(SCFtoGoalsAgainst) + 1)) %>%
            mutate(PointsPercentagesqrt = abs(PointsPercentage)^0.5) %>%
            mutate(CorsiDifftoSOSlog = sign((CF_Per60Team - CA_Per60Team)/SOS) * log(abs((CF_Per60Team - CA_Per60Team)/SOS) + 1)) %>%
            mutate(xGDifftoSOS = (xGF.60 - xGA.60)/SOS) %>% 
            mutate(GStoSOS = GS_mean / SOS) %>%
            mutate(SRStoSOS = SRS/SOS) %>%
            mutate("CF% QoT_min" = sign(allData$"CF% QoT_min") * log(abs(allData$"CF% QoT_min") + 1)) %>%
            mutate(ZSR_min = sign(ZSR_min) * log(abs(ZSR_min) + 1)) %>%
            mutate("Rel CF%_min" = sign(allData$"Rel CF%_min") * log(abs(allData$"Rel CF%_min") + 1)) %>%
            mutate_if(is.numeric, funs(ifelse(is.nan(.), 0,.))) %>%
            mutate_if(is.numeric, funs(ifelse(is.infinite(.), 0,.)))

In [9]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200, scipen = 999)

#...................................Check skewness and kurtosis..................#

kurt = allData %>% select_if(., is.numeric) %>% summarize_all(., funs(moments::kurtosis(., na.rm=TRUE))) %>%
                                         gather(., Variable, Kurtosis)
allData %>% select_if(., is.numeric) %>% summarize_all(., funs(moments::skewness(., na.rm=TRUE))) %>%
                                         gather(., Variable, Skewness) %>%
                                         left_join(., kurt, by = "Variable")
rm(kurt)
allData %>% select_if(., is.numeric) %>% summarize_all(., funs(moments::skewness(., na.rm=TRUE))) %>%
                                          gather(., Variable, Skew) %>%
                                          filter(., Skew >= 1)

Variable,Skewness,Kurtosis
PlayerPoints,0.074724288,3.069980
WeightedGoalieSavePercntage,0.078557224,3.243056
WeightedGPS,0.428811589,3.260166
H2H,0.026571103,2.527590
Q1Record,0.125765210,3.121404
Q2Record,0.318525130,2.894463
Q3Record,0.027757159,2.942516
Q4Record,0.029090232,2.952298
SDRecord,0.088293186,2.651121
AverageAge,0.556504000,3.689177


Variable,Skew
P1_sd,1.019528
GS_sd,1.335249
ZSR_min,1.737566


The preprocessing recipe; mostly straightforward stuff. I remove all zero variance predictors since these variables are inherently useless and present numerical problems (though I don't think we have such variables). Standard center and scaling which is essential for the elastic net (since fitted coefficients are in the units of the response/variable, if the magnitude of the variable is naturally small these coefficients will be "large" and hence be arbitrarily penalized), as well as creating dummy variables for any categorical variables.

Some interaction variables are specified and we use knn imputation with k = 15 nearest neighbours for imputing the missing data from the 2006-2007 seasons as well as the 2018 season.

In [10]:
#.......................Define recipe.............................................#

preProcess.recipe = function(trainX){
  
  mainRecipe = recipe(ResultProper ~., data=trainX) %>%
    step_zv(all_numeric()) %>%
    step_center(all_numeric()) %>%
    step_scale(all_numeric()) %>%
    step_dummy(all_predictors(), -all_numeric()) %>%
    step_zv(all_predictors()) %>%
    step_knnimpute(neighbors = 15, all_numeric(), all_predictors()) %>%
    step_interact(terms = ~ SRS:Fenwick:ELORating) %>%
    step_interact(terms = ~ RegularSeasonWinPercentage:contains("Points")) %>%
    step_interact(terms = ~ FaceoffWinPercentage:ShotPercentage) %>%
    step_interact(terms = ~ contains("Round"):VegasOpeningOdds) %>%
    step_interact(terms = ~ SDRecord:SOS) 
  
  mainRecipe
}

The random search function for hyper parameter tuning; not really a grid search anymore so that's misleading. Randomly chooses an alpha and a lambda to try and replaces the current best values if the resulting model is better.

In [11]:
randomGridSearch = function(iterations, innerTrainX, innerTestX){
  
  score = 0
  alpha.fin = numeric(1)
  lambda.fin = integer(1)
  
  for(m in 1:iterations){
    
    writeLines(paste("Iteration:", m, sep = " "))
    
    alpha_val = as.numeric(runif(1, 0, 1))
    s.lambda_val = as.integer(sample(1:80, 1))
    
    modelX = baggedModel(train = innerTrainX[, !names(innerTrainX) %in% c("ResultProper")], test = innerTestX, 
                       label_train = innerTrainX$ResultProper, alpha.a = alpha_val, s_lambda.a = s.lambda_val)
  
    score.new = roc(response = innerTestX$ResultProper, predictor = modelX$Predictions, levels = c("L", "W"))$auc
    
    if(score.new > score){
      alpha.fin = alpha_val
      lambda.fin = s.lambda_val
      score = score.new
    }
  }
  list(alpha = alpha.fin, lambda = lambda.fin)
}

The model pipe for the outer validation. We split the entire dataset into three folds and save one for the test set; the other two folds form the training set and are sent to the inner pipe. We repeat this so that every fold serves as the test set once.

In [12]:
modelPipe.outer = function(j, folds, lambda.final, alpha.final){
 
  train.param = prep(preProcess.recipe(trainX = allData[-folds[[j]],]), training = allData[-folds[[j]],])
  train = bake(train.param, new_data = allData[-folds[[j]], ])
  test = bake(train.param, new_data = allData[folds[[j]], ])
  
  frameswithPCA = addPCA_variables(traindata = train, testdata = test)
  
  train = frameswithPCA$train
  test = frameswithPCA$test
  
  rm(train.param, frameswithPCA)
  gc()
  
  model = baggedModel(train = train[, !names(train) %in% c("ResultProper")], test=test, label_train = train$ResultProper, 
                                             alpha.a = alpha.final, s_lambda.a = lambda.final)
  
  ROC = roc(response = test$ResultProper, predictor = model$Predictions, levels = c("L", "W"))$auc
  VarImp = model$VariableImportance
  
  list(ROC = ROC, VarImp = VarImp)
}

The subsequent two folds from the outer validation now form the training set. This set is then partioned again with an 80/20 split and we use this partioning only to tune hyper parameters. This is what is referred to as nested cross validation to avoid overly optimistic results.

In [13]:
modelPipe.inner = function(k, folds){
  
  mainTrain = allData[-folds[[k]], ]
  
  innerFolds = createDataPartition(y = mainTrain$ResultProper, times = 1, p = 0.8)

      train.param = prep(preProcess.recipe(trainX = mainTrain[innerFolds[[1]],]), training = mainTrain[innerFolds[[1]],])
      train = bake(train.param, new_data = mainTrain[innerFolds[[1]],])
      test = bake(train.param, new_data = mainTrain[-innerFolds[[1]],])
      
      frameswithPCA = addPCA_variables(traindata = train, testdata = test)
      
      train = frameswithPCA$train
      test = frameswithPCA$test
      
      rm(train.param, frameswithPCA)
      
      results = randomGridSearch(iterations = 130, innerTrainX = train, innerTestX = test)
 
  
  list(alpha = results$alpha, lambda = results$lambda)
  
}

Process variable importance; these metrics should be taken with a grain of salt since the majority of these variables are collinear and if we want to actually gauge the effect of such variables on the probability of winning a NHL series we should use statistical inference and be (much) more careful in specifying our model.

In [14]:
processVarImp = function(varImpRaw, final = FALSE){
  
  varImpNames = varImpRaw %>% 
    select(., contains("Variable")) %>%
    .[,1]
  
  if(final == FALSE){
  
  final = varImpRaw %>% 
                        select(., contains("meanImportance")) %>%
                        transmute(Importance = rowMeans(.)) %>%
                        bind_cols(varImpNames, .)
  }else{
    
  final = varImpRaw %>% 
                        select(., contains("Importance")) %>%
                        transmute(Importance = rowMeans(.)) %>%
                        bind_cols(varImpNames, .)
  }
  
  final
}

The main script that calls the functions above. Does each repeat in parallel.

In [15]:
set.seed(40689)
seeds = sample(1:1000000000, 150, replace = FALSE)

cluster = makeCluster(detectCores())
registerDoParallel(cluster)

results = foreach(p = 1:length(seeds), .combine = "c", .packages = c("tidyverse", "glmnet", "caret", "pROC", "recipes", "fastknn")) %dopar% {
  
  set.seed(seeds[p])
  allFolds = caret::createFolds(y = allData$ResultProper, k = 3)
  
  bestParam = bind_rows(lapply(1:length(allFolds), FUN = modelPipe.inner, folds = allFolds)) 
  finalResults = mapply(FUN = modelPipe.outer, j = 1:length(allFolds), lambda.final = bestParam$lambda, alpha.final = bestParam$alpha, 
                      MoreArgs = list(folds = allFolds), SIMPLIFY = FALSE)

  ROC = mean(unlist(lapply(finalResults, function(x){unlist(x$ROC)})))

  VarImp = processVarImp(varImpRaw = as_tibble(bind_cols(lapply(finalResults, function(x){(x$VarImp)}))), final = FALSE)
  
  write_csv(tibble(ROC = ROC), paste("Iteration_", p, ".csv", sep = ""))
  list(ROC = ROC, VarImp = VarImp)

}

stopCluster(cluster)
rm(cluster)

Displays the results and gives a 95% confidence interval. 

In [16]:
finalROC = unlist(results[c(seq(1, length(results), 2))])
finalVarImp = processVarImp(varImpRaw = results[c(seq(2, length(results),2))] %>% Reduce(bind_cols,.), final = TRUE) 

paste("Final AUROC: ", mean(finalROC), " with a 95% confidence interval given by ", "[", mean(finalROC) - qnorm(0.975)*sd(finalROC)/(length(finalROC)^0.5), ", ", 
      mean(finalROC) + qnorm(0.975)*sd(finalROC)/(length(finalROC)^0.5), "]", sep = "")


[1] "Final AUROC: 0.67512012012012 with a 95% confidence interval given by [0.672597256743428, 0.677642983496813]"

In [19]:
finalVarImp %>% arrange(., -Importance)

Variable,Importance
PC1,0.0440907426325
PC4,0.0052435392782
PowerPlayOppurtunities,0.0044902470944
H2H,0.0044059708862
PenaltyKillPercentage,0.0041934281388
PC2,0.0040802456781
logofSOG,0.0035307133044
SOG_Last20,0.0033670346046
PC3,0.0033408782594
WeightedGPS,0.0033219918468
